In [ ]:
from db.db import backup_database

backup_destination = r"N:\PPDO\BS\Records Department\Archive_Data"
backup_database(backup_dir=backup_destination, compress=True)

In [ ]:
from sqlalchemy.orm import Session
from sqlalchemy import or_
import pandas as pd
import numpy as np
from db import get_db_engine
from db.models import FileLocation, File, FileContent
from text_extraction.pdf_extraction import PDFTextExtractor
from embedding.minilm import MiniLMEmbedder
from utils import extract_server_dirs, build_file_path
server_mount = r"N:\PPDO\Records"
target_filepath = r"N:\PPDO\Records\23xx   Crown College (College 3)\2318\2318\H - Submittals\Lumber_ProductData.pdf"
target_location = r"N:\PPDO\Records\37xx   Oakes College\3701\3701"


pdf_extractor = PDFTextExtractor()
embedder = MiniLMEmbedder()

text = pdf_extractor(target_filepath)
embeddings = embedder.encode(text)
target_loc_dirs = extract_server_dirs(full_path=target_location, base_mount=server_mount)
engine = get_db_engine()

target_sim_distance = lambda file_embeds: 1 - np.dot(file_embeds, embeddings)
build_local_path = lambda row: build_file_path(base_mount=server_mount, server_dir=row['file_server_directories'], filename=row['filename'])

with Session(engine) as db_session:
    files_located_in_dir = or_(
        FileLocation.file_server_directories == target_loc_dirs,
        FileLocation.file_server_directories.startswith(target_loc_dirs + '/')
    )

    query = db_session.query(FileLocation, FileContent)

In [1]:
from sqlalchemy.orm import Session
from db import get_db_engine
from db.models import PathPattern, Base
from utils import extract_server_dirs

def add_exclusion_pattern():
    # Full path to exclude
    full_path = r"N:\PPDO\Records\16xx   Cowell College\1639\1639\G - Construction\G22 - Legal Claims and Public Records Act Request\Provided to Legal 2.5.13 FTP"
    base_mount = r"N:\PPDO\Records"
    
    # Convert to server-relative path format (the format stored in the database)
    server_relative_path = extract_server_dirs(full_path, base_mount)
    
    # Create pattern that will match this directory and all contents
    pattern = f"{server_relative_path}/*"
    
    print(f"Creating exclusion pattern: {pattern}")
    
    # Connect to database
    engine = get_db_engine()
    
    with Session(engine) as session:
        # Check if pattern already exists
        existing = session.query(PathPattern).filter_by(pattern=pattern).first()
        if existing:
            print(f"Pattern already exists with ID {existing.id}")
            return
        
        # Create new pattern with both embedding and tagging contexts
        new_pattern = PathPattern(
            pattern=pattern,
            pattern_type="directory",
            treatment="exclude",
            description="Legal claims and public records request documents",
            contexts=["add_files_embedding", "add_files_tagging"],
            enabled=True
        )
        
        session.add(new_pattern)
        session.commit()
        print(f"Exclusion pattern added with ID {new_pattern.id}")

# Execute the function
add_exclusion_pattern()

2025-09-23 15:54:29,268 db.db INFO Creating database engine


Creating exclusion pattern: 16xx   Cowell College/1639/1639/G - Construction/G22 - Legal Claims and Public Records Act Request/Provided to Legal 2.5.13 FTP/*
Exclusion pattern added with ID 1
